<a href="https://colab.research.google.com/github/DavidS41/Li_DSPN_S25/blob/main/Exercise%2010%20mixed-effects-models%20dsli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 10: Mixed effects

1. Loading and formatting the data 1/1
2. Model fitting 4/4
3. Model assessment 4/4
4. Reflection 1/1

This homework assignment is designed to give you practice fitting and interpreting mixed effects models.

We will be using the **LexicalData.csv** and **Items.csv** files from the *Homework/lexDat* folder in the class GitHub repository again.

This data is a subset of the [English Lexicon Project database](https://elexicon.wustl.edu/). It provides the reaction times (in milliseconds) of many subjects as they are presented with letter strings and asked to decide, as quickly and as accurately as possible, whether the letter string is a word or not. The **Items.csv** provides characteristics of the words used, namely frequency (how common is this word?) and length (how many letters?). Unlike in the previous homework, there isn't any missing data in the **LexicalData.csv** file.

*Data courtesy of Balota, D.A., Yap, M.J., Cortese, M.J., Hutchison, K.A., Kessler, B., Loftis, B., Neely, J.H., Nelson, D.L., Simpson, G.B., & Treiman, R. (2007). The English Lexicon Project. Behavior Research Methods, 39, 445-459.*

---
## 1. Loading and formatting the data (1 point)

Load in data from the **LexicalData.csv** and **Items.csv** files. As in the previous homeworks, remove the commas from the reaction times and convert them from strings to numbers. Use `left_join` to add word characteristics `Length` and `Log_Freq_Hal` from **Items** to **LexicalData**.

*Note: the `Freq_HAL` variable in **Items.csv** has a similar formatting issue, using string values with commas. We're not going to worry about fixing this since we're only using `Log_Freq_HAL`, which is the natural log transformation of `Freq_HAL`, in this homework.*

In [ ]:
# WRITE YOUR CODE HERE
#Load tidyverse
#install.packages("tidyverse")
library(tidyverse)
library(ggplot2)

#Load the data
system("gdown --id 1GNeEukTniu54HAquNk4IZ3FQDjUaPMft")
list.files()
d_lex <- read.csv("LexicalData.csv")
#head(d_lex)

system("gdown --id 1m0gm26jgxnI8Jlkkc3RN8UYnxeBOs072")
list.files()
d_items<- read.csv("Items.csv")
#head(d_items)
d_items_2<- d_items%>%
  select(Word, Length, Log_Freq_HAL)%>%
  rename(D_Word = Word)
#head(d_items_2)

d1<-left_join(d_lex, d_items_2, by = "D_Word")%>%
  drop_na()

d1$D_RT <- gsub(",", "", d1$D_RT)
d1$D_RT <- as.numeric(d1$D_RT)

head(d1)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


[1] "LexicalData.csv" "sample_data"

[1] "Items.csv"       "LexicalData.csv" "sample_data"

,Sub_ID,Trial,Type,D_RT,D_Word,Outlier,D_Zscore,Length,Log_Freq_HAL
,<int>,<int>,<int>,<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>
1,157,1,1,710,browse,false,-0.437,6,8.856
2,67,1,1,1094,refrigerant,false,0.825,11,4.644
3,120,1,1,587,gaining,false,-0.645,7,8.304
4,21,1,1,984,cheerless,false,0.025,9,2.639
5,236,1,1,577,pattered,false,-0.763,8,1.386
6,236,2,1,715,conjures,false,-0.364,8,5.268


---
## 2. Model fitting (4 points)

First, fit a linear model with `Log_Freq_HAL` and `Length` as predictors, and `D_RT` as the output. Include an interaction term. Use `summary()` to look at the model output.

In [ ]:
# WRITE YOUR CODE HERE
lin_mod<-lm(D_RT ~ Log_Freq_HAL*Length, data = d1)
summary(lin_mod)


Call:
lm(formula = D_RT ~ Log_Freq_HAL * Length, data = d1)

Residuals:
     Min       1Q   Median       3Q      Max 
-1118.01  -205.23   -86.95    90.77  3147.07 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)         610.1903    14.6678  41.601  < 2e-16 ***
Log_Freq_HAL         -6.0239     1.9678  -3.061  0.00221 ** 
Length               47.7531     1.6368  29.175  < 2e-16 ***
Log_Freq_HAL:Length  -2.9421     0.2348 -12.528  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 359.1 on 62606 degrees of freedom
Multiple R-squared:  0.09473,	Adjusted R-squared:  0.09469 
F-statistic:  2184 on 3 and 62606 DF,  p-value: < 2.2e-16


Now, install `lme4` using `install.packages()` and then load the library.

In [ ]:
# WRITE YOUR CODE HERE
install.packages("lme4")
library(lme4)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘reformulas’, ‘RcppEigen’


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack




Now fit a mixed effects model that includes the same predictors as the linear model above, as well as random intercepts for `Sub_ID` (i.e., cases where subject ID shifts the RT mean). Use `summary()` to look at the model output.

In [ ]:
# WRITE YOUR CODE HERE

mixed_effects<-lmer(D_RT ~ Log_Freq_HAL*Length + (1 | Sub_ID), data=d1)
summary(mixed_effects)

Linear mixed model fit by REML ['lmerMod']
Formula: D_RT ~ Log_Freq_HAL * Length + (1 | Sub_ID)
   Data: d1

REML criterion at convergence: 888235.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5058 -0.5472 -0.1568  0.3103 10.7381 

Random effects:
 Groups   Name        Variance Std.Dev.
 Sub_ID   (Intercept) 46333    215.3   
 Residual             82978    288.1   
Number of obs: 62610, groups:  Sub_ID, 299

Fixed effects:
                    Estimate Std. Error t value
(Intercept)         616.8445    17.1522  35.963
Log_Freq_HAL         -7.4374     1.5830  -4.698
Length               47.7477     1.3162  36.277
Log_Freq_HAL:Length  -2.8778     0.1888 -15.239

Correlation of Fixed Effects:
            (Intr) Lg_F_HAL Length
Log_Frq_HAL -0.645                
Length      -0.656  0.917         
Lg_Fr_HAL:L  0.582 -0.942   -0.923

---
## 3. Model assessment (4 points)

Compare the three t-values for the fixed effects and the mixed effects models. How do they differ, and why?

> *Write your response here*
> The three t-values for: Log_Freq_HAL, Length, and the interaction term: Log_Freq_HAL:Length in the fixed effects modelare -3.061, 29.175, and -12.528 respectively. The t-values for these three (in the same order) for the mixed effects model are: -4.698, 36.277, -15.239. Generally the magnitude of the t-values are larger suggesting that the coefficient estimates are more significant. This is likely because the mixed effects model accounts for some variance/noise that the fixed effects model cannot and the relationship between the predictors and DV (RT) is more clear within each group of the random effect (within observations of the same subject).

Use the Aikeke Information Criterion (AIC) to compare these two models. Which one is better?

In [ ]:
# WRITE YOUR CODE HERE
ic = AIC(lin_mod, mixed_effects)
ic
diff(ic$AIC)


,df,AIC
,<dbl>,<dbl>
lin_mod,5,914436.4
mixed_effects,6,888247.6


[1] -26188.82

> *Write your response here*
> The mixed effects model has the lowest AIC suggesting that it's the better fitting model (compared to the basic linear model) -- it explains more variance even when penalizing for the additional complexity of incorporating the random effect.

---
##  4. Reflection (1 point)

What other random effects could be controlled for in this data set?

> *Write your response here*
> You could also incorporate random effects for Trial	as you may not expect the difference between the trial numbers to map to differences in reaction time/you largely expect some random effect from different trials. Maybe you could incorporate one for D_Word if you reduce the dataset to focus only on some subset of words that are similar in length (you wouldn't necessarily expect the same length words to take longer/shorter to be recognized and any difference you observe wouldn't map from the category difference to a real world difference).

**DUE:** 5pm EST, March 18, 2024

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here.
> *Someone's Name*